<a href="https://colab.research.google.com/github/emerson1000/notebooks/blob/main/Cola_CuelloBotella_Servidor_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cola, cuello de botella, servidores, medidas de eficiencia, tiempos de servicios y fuga de clientes**

### Cola (Queue)

Contexto: Imagina que estás desarrollando un programa para gestionar las solicitudes de atención al cliente en una línea de soporte técnico. Necesitas utilizar una estructura de datos para organizar y administrar las solicitudes de manera que se atiendan en el orden en que se reciben.

In [ ]:
from queue import Queue

# Crear una cola
cola = Queue()

# Agregar elementos a la cola
cola.put(1)
cola.put(2)
cola.put(3)

# Obtener y eliminar elementos de la cola (FIFO)
elemento = cola.get()
print("Elemento retirado de la cola:", elemento)

Elemento retirado de la cola: 1


In [ ]:
# Obtener y eliminar elementos de la cola (FIFO)
elemento = cola.get()
print("Elemento retirado de la cola:", elemento)

Elemento retirado de la cola: 2


In [ ]:
# Obtener y eliminar elementos de la cola (FIFO)
elemento = cola.get()
print("Elemento retirado de la cola:", elemento)

Elemento retirado de la cola: 3


Otro ejemplo,  consiste en que se simule una línea de producción donde los trabajos se acumulan en una cola antes de ser procesados por diferentes estaciones de trabajo. Utilizaremos la biblioteca SimPy para llevar a cabo esta simulación.

In [ ]:
!pip install simpy

In [ ]:
import simpy
import random

# Definimos una función para representar una estación de trabajo
def estacion_trabajo(env, nombre, tiempo_procesamiento):
    while True:
        # Esperamos a que llegue un trabajo a la cola
        yield env.timeout(1)

        # La estación de trabajo está disponible, procesa un trabajo
        print(f'{nombre} comienza a procesar en el tiempo {env.now}')
        yield env.timeout(tiempo_procesamiento)
        print(f'{nombre} termina de procesar en el tiempo {env.now}')

# Configuramos la simulación
env = simpy.Environment()

# Creamos una cola de trabajos
cola_trabajos = simpy.Store(env)

# Creamos estaciones de trabajo con diferentes tiempos de procesamiento
estaciones = [
    ('Estación A', 2),  # Estación A procesa trabajos en 2 unidades de tiempo
    ('Estación B', 3),  # Estación B procesa trabajos en 3 unidades de tiempo
    ('Estación C', 4)   # Estación C procesa trabajos en 4 unidades de tiempo
]

# Creamos las estaciones de trabajo como procesos
for nombre, tiempo_procesamiento in estaciones:
    env.process(estacion_trabajo(env, nombre, tiempo_procesamiento))

# Función para generar trabajos y colocarlos en la cola
def generar_trabajos(env):
    for i in range(10):
        tiempo_llegada = random.randint(1, 5)  # Tiempo aleatorio entre 1 y 5 unidades de tiempo
        yield env.timeout(tiempo_llegada)
        trabajo = f'Trabajo {i + 1} llega en {tiempo_llegada} unidades de tiempo'
        print(trabajo)
        cola_trabajos.put(trabajo)

# Procesamos los trabajos de la cola
def procesar_trabajos(env):
    while True:
        trabajo = yield cola_trabajos.get()
        estacion_elegida, tiempo_procesamiento = random.choice(estaciones)
        env.process(estacion_trabajo(env, estacion_elegida, tiempo_procesamiento))

# Ejecutamos la simulación durante un período de tiempo
env.process(generar_trabajos(env))
env.process(procesar_trabajos(env))
env.run(until=11)

Estación A comienza a procesar en el tiempo 1
Estación B comienza a procesar en el tiempo 1
Estación C comienza a procesar en el tiempo 1
Trabajo 1 llega en 2 unidades de tiempo
Estación A termina de procesar en el tiempo 3
Estación B comienza a procesar en el tiempo 3
Estación B termina de procesar en el tiempo 4
Estación A comienza a procesar en el tiempo 4
Estación C termina de procesar en el tiempo 5
Estación B comienza a procesar en el tiempo 5
Estación B termina de procesar en el tiempo 6
Estación A termina de procesar en el tiempo 6
Estación C comienza a procesar en el tiempo 6
Trabajo 2 llega en 5 unidades de tiempo
Estación B comienza a procesar en el tiempo 7
Estación A comienza a procesar en el tiempo 7
Estación B termina de procesar en el tiempo 8
Trabajo 3 llega en 1 unidades de tiempo
Estación C comienza a procesar en el tiempo 8
Estación A termina de procesar en el tiempo 9
Estación B comienza a procesar en el tiempo 9
Trabajo 4 llega en 1 unidades de tiempo
Estación B c

En este ejemplo:

* Hemos definido una función estacion_trabajo que representa una estación de trabajo que procesa trabajos.

* Hemos creado una cola (cola_trabajos) donde los trabajos esperan a ser procesados.

* Generamos trabajos de manera aleatoria y los colocamos en la cola.

* Procesamos los trabajos de la cola asignándolos a una estación de trabajo disponible.

* Finalmente, ejecutamos la simulación durante un período de tiempo de 10 unidades.

La simulación mostrará cómo los trabajos se acumulan en la cola antes de ser procesados por las estaciones de trabajo. Observarás que los trabajos esperan en la cola si todas las estaciones están ocupadas, lo que es un ejemplo de cómo las colas pueden formarse en un proceso de producción cuando las estaciones de trabajo no pueden procesar los trabajos de manera inmediata. Puedes ajustar los tiempos de procesamiento de las estaciones y los tiempos de llegada de los trabajos para experimentar con diferentes escenarios y observar cómo afectan la formación de colas en el proceso.

**Inicio de la simulación:**
Las estaciones A, B y C comienzan a procesar en el tiempo 1. Esto significa que están disponibles y esperando trabajos.


* Procesamiento inicial
* Llegada de más trabajos:
* Finalización de la simulación
* La simulación se detiene en el tiempo dado (until=11).



En cuanto a mostrar las "colas" del proceso, podemos realizar un seguimiento de los trabajos en cola en cada estación de trabajo. Aquí hay un algoritmo que muestra las colas en función de los trabajos que están esperando ser procesados en cada estación de trabajo:

In [ ]:
# Crear diccionarios para rastrear las colas en cada estación
colas = {nombre: [] for nombre, _ in estaciones}

# Función para mostrar las colas en cada estación
def mostrar_colas():
    for nombre, _ in estaciones:
        print(f'Cola en {nombre}: {", ".join(colas[nombre])}')

# Modificar la función de procesar_trabajos para actualizar las colas
def procesar_trabajos(env):
    while True:
        trabajo = yield cola_trabajos.get()
        estacion_elegida, tiempo_procesamiento = random.choice(estaciones)
        cola_trabajos.put(trabajo)  # Volver a poner el trabajo en la cola
        colas[estacion_elegida].append(trabajo)  # Agregar trabajo a la cola de la estación
        env.process(estacion_trabajo(env, estacion_elegida, tiempo_procesamiento))

# Ejecutamos la simulación durante un período de tiempo
env.process(generar_trabajos(env))
env.process(procesar_trabajos(env))
env.run(until=15)

# Mostrar las colas al final de la simulación
mostrar_colas()


Estación C comienza a procesar en el tiempo 11
Estación B comienza a procesar en el tiempo 11
Estación C comienza a procesar en el tiempo 11
Estación C termina de procesar en el tiempo 12
Estación B termina de procesar en el tiempo 12
Estación B termina de procesar en el tiempo 12
Estación A termina de procesar en el tiempo 12
Estación A termina de procesar en el tiempo 12
Trabajo 1 llega en 1 unidades de tiempo
Estación C comienza a procesar en el tiempo 13
Estación B comienza a procesar en el tiempo 13
Estación B comienza a procesar en el tiempo 13
Estación A comienza a procesar en el tiempo 13
Estación A comienza a procesar en el tiempo 13
Estación B comienza a procesar en el tiempo 13
Trabajo 6 llega en 4 unidades de tiempo
Estación B termina de procesar en el tiempo 14
Cola en Estación A: 
Cola en Estación B: 
Cola en Estación C: Trabajo 6 llega en 4 unidades de tiempo


Este código modificará la función procesar_trabajos para que vuelva a colocar el trabajo en la cola después de seleccionar una estación de trabajo. También se agrega el trabajo a la cola de la estación correspondiente. Al final de la simulación, se muestra el contenido de las colas en cada estación de trabajo. Esto te dará una idea de cómo se forman las colas en cada estación a lo largo del tiempo.

### Cuellos de Botella (Bottlenecks)

Contexto: Eres un ingeniero de procesos en una fábrica de productos electrónicos. Quieres analizar el proceso de ensamblaje para identificar si hay algún punto que ralentiza la producción y afecta el tiempo total de ensamblaje.

In [ ]:
# Tiempos de procesamiento en una línea de ensamblaje (en segundos)
tiempos_procesamiento = [10, 5, 8, 12, 6]

# Calcular el tiempo total de procesamiento
tiempo_total = sum(tiempos_procesamiento)
print("Tiempo total de procesamiento:", tiempo_total)

# Identificar el cuello de botella (máximo tiempo de procesamiento)
cuello_de_botella = max(tiempos_procesamiento)
print("Cuello de botella:", cuello_de_botella)

Tiempo total de procesamiento: 41
Cuello de botella: 12


Otro ejemplo aplicado al área industrial de procesos en Python relacionado con "Cuellos de botellas", podría involucrar la simulación de un proceso de producción que identifica y analiza cuellos de botella. En este escenario, utilizaremos la biblioteca SimPy para simular un proceso de producción en el que varias estaciones de trabajo procesan trabajos, y luego calcularemos la eficiencia del proceso para identificar posibles cuellos de botella:

In [ ]:
!pip install simpy

In [ ]:
import simpy
import random

# Definimos una función para representar una estación de trabajo
def estacion_trabajo(env, nombre, tiempo_procesamiento):
    while True:
        # Simulamos el tiempo que lleva procesar un trabajo
        yield env.timeout(tiempo_procesamiento)

        # La estación de trabajo ha completado el trabajo
        print(f'{nombre} ha terminado en el tiempo {env.now}')

# Configuramos la simulación
env = simpy.Environment()

# Creamos un proceso de producción con varias estaciones de trabajo
estaciones = [
    ('Estación A', 2),  # Estación A procesa trabajos en 2 unidades de tiempo
    ('Estación B', 3),  # Estación B procesa trabajos en 3 unidades de tiempo
    ('Estación C', 4)   # Estación C procesa trabajos en 4 unidades de tiempo
]

# Creamos las estaciones de trabajo como procesos
for nombre, tiempo_procesamiento in estaciones:
    env.process(estacion_trabajo(env, nombre, tiempo_procesamiento))

# Ejecutamos la simulación durante un período de tiempo
env.run(until=10)

# Calculamos la eficiencia del proceso
total_tiempo_produccion = sum(tiempo_procesamiento for _, tiempo_procesamiento in estaciones)
eficiencia_proceso = total_tiempo_produccion / env.now

# Mostramos los resultados
print(f'Tiempo total de producción: {total_tiempo_produccion}')
print(f'Tiempo de simulación: {env.now}')
print(f'Eficiencia del proceso: {eficiencia_proceso:.2%}')

Estación A ha terminado en el tiempo 2
Estación B ha terminado en el tiempo 3
Estación C ha terminado en el tiempo 4
Estación A ha terminado en el tiempo 4
Estación B ha terminado en el tiempo 6
Estación A ha terminado en el tiempo 6
Estación C ha terminado en el tiempo 8
Estación A ha terminado en el tiempo 8
Estación B ha terminado en el tiempo 9
Tiempo total de producción: 9
Tiempo de simulación: 10
Eficiencia del proceso: 90.00%


En este ejemplo:

Definimos una función estacion_trabajo que simula el tiempo que lleva procesar un trabajo en una estación de trabajo.

Configuramos una simulación en la que hay tres estaciones de trabajo con diferentes tiempos de procesamiento.

Ejecutamos la simulación durante un período de tiempo específico (en este caso, 10 unidades de tiempo).

Calculamos la eficiencia del proceso dividiendo el tiempo total de producción entre el tiempo de simulación. La eficiencia se expresa como un porcentaje.

El resultado de la simulación mostrará cómo las estaciones de trabajo procesan los trabajos y qué estación podría ser un cuello de botella según la eficiencia del proceso calculada. Puedes ajustar los tiempos de procesamiento de las estaciones de trabajo para experimentar con diferentes escenarios y analizar cómo afectan la eficiencia del proceso.

La **eficiencia** del proceso se interpreta como la relación entre el tiempo total de producción y el tiempo de simulación. En otras palabras, es una medida de qué tan eficiente es tu proceso de producción en términos de tiempo. Una eficiencia del proceso del 90.00% significa que durante el período de simulación, el 90.00% del tiempo se utilizó realmente para la producción, mientras que el 10.00% del tiempo se desperdició o se dedicó a otros fines (por ejemplo, esperando o en inactividad).

La eficiencia del proceso parece disminuir cuando aumentas la cantidad de simulaciones, es importante entender que la eficiencia del proceso se calcula en función del tiempo real de producción y el tiempo de simulación. En la simulación, el tiempo de simulación es de 10 unidades de tiempo, independientemente de cuántas estaciones de trabajo haya.

Cuando agregas más estaciones de trabajo al proceso, es probable que los trabajos se completen más rápidamente, ya que hay más capacidad para procesarlos. Como resultado, el tiempo total de producción disminuye, lo que aumenta la eficiencia del proceso. Esto puede llevar a la percepción de que la eficiencia disminuye a medida que aumentas la cantidad de simulaciones.

Sin embargo, es importante tener en cuenta que una mayor eficiencia generalmente es deseable en un proceso de producción, ya que significa que se está utilizando mejor el tiempo de trabajo. La disminución en la eficiencia al aumentar la cantidad de simulaciones puede ser un efecto de la velocidad a la que se pueden procesar los trabajos en un sistema más eficiente. Por lo tanto, en términos de eficiencia, una mayor eficiencia generalmente es preferible a una menor.

### Servidores

Contexto: Estás simulando el funcionamiento de un restaurante y deseas modelar cómo los camareros toman pedidos de los clientes y sirven la comida. Quieres mostrar cómo los servidores desempeñan un papel importante en la experiencia de los clientes.

In [ ]:
class Camarero:
    def __init__(self, nombre):
        self.nombre = nombre

    def tomar_pedido(self, cliente, pedido):
        print(f"{self.nombre} toma el pedido de {cliente}: {pedido}")

    def servir_comida(self, cliente, plato):
        print(f"{self.nombre} sirve a {cliente}: {plato}")

# Crear servidores
camarero1 = Camarero("Juan")
camarero2 = Camarero("María")

# Simular el proceso de tomar pedido y servir comida
cliente1 = "Cliente 1"
pedido1 = "Pizza"
camarero1.tomar_pedido(cliente1, pedido1)
camarero1.servir_comida(cliente1, pedido1)

Juan toma el pedido de Cliente 1: Pizza
Juan sirve a Cliente 1: Pizza


Otro ejemplo en Python relacionado con servidores en un proceso industrial, podemos crear un simulador de un sistema de control de producción que utiliza un servidor para coordinar el flujo de trabajo en una línea de producción. En este escenario, el servidor asignará trabajos a diferentes estaciones de trabajo según su disponibilidad:

In [ ]:
!pip install simpy

In [ ]:
import simpy
import random

# Definimos una función para representar una estación de trabajo
def estacion_trabajo(env, nombre, servidor):
    while True:
        yield env.timeout(random.uniform(1, 3))  # Tiempo de procesamiento simulado

        # La estación de trabajo está disponible, solicita un trabajo al servidor
        print(f'{nombre} solicita trabajo en {env.now}')
        with servidor.request() as req:
            yield req

            # Procesamos el trabajo
            print(f'{nombre} comienza a procesar en {env.now}')
            yield env.timeout(random.uniform(1, 3))  # Procesamiento simulado
            print(f'{nombre} termina de procesar en {env.now}')

# Configuramos la simulación
env = simpy.Environment()

# Creamos un servidor que coordina las estaciones de trabajo
servidor = simpy.Resource(env, capacity=1)  # Capacidad de 1 para simular un servidor único

# Creamos varias estaciones de trabajo
estacion1 = env.process(estacion_trabajo(env, 'Estación 1', servidor))
estacion2 = env.process(estacion_trabajo(env, 'Estación 2', servidor))
estacion3 = env.process(estacion_trabajo(env, 'Estación 3', servidor))

# Ejecutamos la simulación durante un período de tiempo
env.run(until=10)

Estación 3 solicita trabajo en 1.0274178085667616
Estación 3 comienza a procesar en 1.0274178085667616
Estación 1 solicita trabajo en 2.51804015952455
Estación 2 solicita trabajo en 2.6095478146379483
Estación 3 termina de procesar en 3.559013603540909
Estación 1 comienza a procesar en 3.559013603540909
Estación 3 solicita trabajo en 6.0622362634595
Estación 1 termina de procesar en 6.217403435920677
Estación 2 comienza a procesar en 6.217403435920677
Estación 2 termina de procesar en 8.506985477291973
Estación 3 comienza a procesar en 8.506985477291973
Estación 1 solicita trabajo en 9.205185994638033


En este ejemplo:

Definimos una función estacion_trabajo que representa una estación de trabajo. Cada estación de trabajo tiene un tiempo de procesamiento simulado y solicita trabajos al servidor cuando está disponible.

Configuramos la simulación con SimPy y creamos un servidor utilizando la clase simpy.Resource con una capacidad de 1 para simular un servidor único.

Creamos tres estaciones de trabajo (estacion1, estacion2, y estacion3) como procesos en la simulación.

Ejecutamos la simulación durante un período de tiempo definido (env.run(until=20)).

Este ejemplo ilustra cómo un servidor puede coordinar las estaciones de trabajo en un proceso industrial, asignando trabajos a las estaciones disponibles. Puedes ajustar los parámetros y tiempos de procesamiento según tus necesidades específicas para simular un proceso industrial más realista. Además, puedes agregar métricas e indicadores de rendimiento para analizar el rendimiento del proceso y la eficiencia del servidor.

El código es una simulación de estaciones de trabajo en un proceso industrial utilizando la biblioteca SimPy en Python. Vamos a analizar el código paso a paso:

* Definición de la función estacion_trabajo:

Esta función representa una estación de trabajo en el proceso industrial. La estación de trabajo tiene un nombre (nombre) y utiliza la biblioteca simpy para simular su comportamiento. El comportamiento de la estación se modela como un bucle infinito donde:

yield env.timeout(random.uniform(1, 3)): Se simula un tiempo de procesamiento aleatorio entre 1 y 3 unidades de tiempo. Esto representa el tiempo que lleva procesar un trabajo en la estación de trabajo.

print(f'{nombre} solicita trabajo en {env.now}'): Se muestra un mensaje indicando que la estación de trabajo está solicitando un trabajo al servidor en un momento específico (env.now representa el tiempo de la simulación).

with servidor.request() as req:: Se solicita un recurso del servidor (en este caso, la estación de trabajo está solicitando acceso al servidor).

yield req: Se espera hasta que el servidor le conceda acceso. Mientras tanto, la estación de trabajo está en espera.

print(f'{nombre} comienza a procesar en {env.now}'): Se muestra un mensaje indicando que la estación de trabajo comenzó a procesar un trabajo en un momento específico.

yield env.timeout(random.uniform(1, 3)): Se simula el tiempo de procesamiento del trabajo, nuevamente con un valor aleatorio.

print(f'{nombre} termina de procesar en {env.now}'): Se muestra un mensaje indicando que la estación de trabajo terminó de procesar un trabajo en un momento específico.

* Configuración de la simulación:

env = simpy.Environment(): Se crea un entorno de simulación utilizando SimPy. Esto controlará el tiempo y los eventos en la simulación.

servidor = simpy.Resource(env, capacity=1): Se crea un recurso (en este caso, el servidor) con una capacidad de 1. Esto simula un servidor único que coordina las estaciones de trabajo. Las estaciones solicitarán acceso a este recurso para procesar trabajos.

Se crean tres procesos estacion1, estacion2 y estacion3 que representan tres estaciones de trabajo utilizando la función estacion_trabajo. Cada estación de trabajo tiene un nombre y accede al mismo servidor.

* Ejecución de la simulación:

env.run(until=20): Se ejecuta la simulación durante un período de tiempo de 20 unidades de tiempo. Durante este tiempo, las estaciones de trabajo solicitarán y procesarán trabajos utilizando el servidor.
El resultado de la simulación mostrará mensajes que indican cuándo cada estación de trabajo solicita y completa trabajos, lo que te permitirá observar cómo se coordina el flujo de trabajo en el proceso industrial.

### Políticas de Atención (Service Policies)

Contexto: Eres el administrador de un hospital y necesitas establecer una política de atención para los pacientes que ingresan al hospital. Quieres garantizar que los pacientes se atiendan de manera apropiada, con prioridad para los casos más graves.

In [ ]:
class Hospital:
    def __init__(self):
        self.pacientes_graves = []
        self.pacientes_normales = []

    def ingresar_paciente(self, paciente, gravedad):
        if gravedad == "grave":
            self.pacientes_graves.append(paciente)
        else:
            self.pacientes_normales.append(paciente)

    def atender_pacientes(self):
        for paciente in self.pacientes_graves:
            print(f"Atendiendo al paciente grave: {paciente}")
        for paciente in self.pacientes_normales:
            print(f"Atendiendo al paciente normal: {paciente}")

# Crear un hospital
hospital = Hospital()

# Ingresar pacientes con diferentes niveles de gravedad
hospital.ingresar_paciente("Paciente A", "grave")
hospital.ingresar_paciente("Paciente B", "normal")
hospital.ingresar_paciente("Paciente C", "grave")

# Aplicar la política de atención
hospital.atender_pacientes()

Atendiendo al paciente grave: Paciente A
Atendiendo al paciente grave: Paciente C
Atendiendo al paciente normal: Paciente B


### Medidas de Eficiencia - Tiempos de servicio - Fuga de clientes


Supongamos que eres el gerente de una tienda minorista y quieres evaluar la eficiencia de tu línea de cajas registradoras. Tienes tres cajas registradoras y deseas analizar cuánto tiempo pasa un cliente esperando en la cola y cuántos clientes pueden abandonar debido a largos tiempos de espera. Tu objetivo es optimizar la eficiencia del proceso de pago.

Desarrollo en Python:

Primero, definiremos algunas variables y simularemos el proceso de pago en la tienda durante un período de tiempo. Luego, calcularemos las medidas de eficiencia, los tiempos de servicio y la fuga de clientes.

In [ ]:
import random

# Variables del ejemplo
numero_cajas = 10
tiempo_simulacion_horas = 3
clientes_atendidos = 0
clientes_abandonaron = 0
tiempo_total_servicio = 0


In [ ]:
# Simulación del proceso de pago
for hora in range(tiempo_simulacion_horas):
    for caja in range(numero_cajas):
        if random.random() < 0.8:  # 80% de probabilidad de que un cliente llegue
            tiempo_servicio = random.uniform(1, 5)  # Tiempo de servicio entre 1 y 5 minutos
            tiempo_total_servicio += tiempo_servicio
            clientes_atendidos += 1
            if tiempo_servicio > 4:  # Si el tiempo de servicio es largo, el cliente abandona
                clientes_abandonaron += 1

In [ ]:
# Medidas de eficiencia
tiempo_promedio_servicio = tiempo_total_servicio / clientes_atendidos
tiempo_promedio_espera = tiempo_promedio_servicio / numero_cajas

In [ ]:
# Resultados e interpretación
print(f"Clientes atendidos: {clientes_atendidos}")
print(f"Clientes que abandonaron debido a largos tiempos de espera: {clientes_abandonaron}")
print(f"Tiempo promedio de servicio por cliente: {tiempo_promedio_servicio:.2f} minutos")
print(f"Tiempo promedio de espera por cliente: {tiempo_promedio_espera:.2f} minutos")

Clientes atendidos: 24
Clientes que abandonaron debido a largos tiempos de espera: 7
Tiempo promedio de servicio por cliente: 3.24 minutos
Tiempo promedio de espera por cliente: 0.32 minutos


Hemos simulado el proceso de pago en la tienda durante un período de 3 horas con 10 cajas registradoras. Los clientes llegan con una probabilidad del 80%, y el tiempo de servicio se selecciona al azar entre 1 y 5 minutos.

Calculamos el número de clientes atendidos y el número de clientes que abandonaron debido a largos tiempos de espera (más de 4 minutos en este caso).

Calculamos el tiempo promedio de servicio por cliente y el tiempo promedio de espera por cliente.

Estos resultados nos permiten evaluar la eficiencia del proceso de pago. Por ejemplo, si muchos clientes están abandonando debido a largos tiempos de espera, esto podría indicar un problema de eficiencia que requiere atención.

Este ejemplo demuestra cómo se pueden utilizar medidas de eficiencia, tiempos de servicio y la fuga de clientes para evaluar y mejorar un proceso en un entorno de negocio real.

Otro ejemplo:

Supongamos que gestionamos una fábrica de productos electrónicos. Tenemos una línea de ensamblaje donde los trabajadores ensamblan componentes en productos terminados. Nos preocupa la eficiencia del proceso, el tiempo de servicio y la posibilidad de que los trabajadores se frustren y abandonen el trabajo debido a condiciones laborales inadecuadas.

Problematica: Queremos evaluar la eficiencia de nuestra línea de ensamblaje, medir el tiempo que lleva ensamblar cada producto y controlar la fuga de trabajadores debido a condiciones de trabajo desfavorables.

Para abordar este problema, podemos simular la línea de ensamblaje y recopilar datos sobre el tiempo que lleva ensamblar productos y el número de trabajadores que abandonan el proceso debido a tiempos de servicio prolongados o condiciones de trabajo insatisfactorias. Luego, calcularemos medidas de eficiencia y analizaremos los resultados.

In [ ]:
import simpy
import random

# Configuración de la simulación
env = simpy.Environment()
tiempo_total_simulacion = 480  # Duración total de la simulación en minutos (8 horas)
productos_ensamblados = 0
trabajadores_abandonados = 0

# Parámetros de la línea de ensamblaje
tiempo_ensamblaje_min = 10  # Tiempo mínimo para ensamblar un producto en minutos
tiempo_ensamblaje_max = 30  # Tiempo máximo para ensamblar un producto en minutos
condiciones_laborales_insatisfactorias = 0.2  # Probabilidad de condiciones laborales insatisfactorias

# Función para el ensamblaje de productos
def ensamblar_producto(env, trabajador):
    global productos_ensamblados, trabajadores_abandonados
    while True:
        tiempo_ensamblaje = random.uniform(tiempo_ensamblaje_min, tiempo_ensamblaje_max)
        yield env.timeout(tiempo_ensamblaje)
        productos_ensamblados += 1
        if random.random() < condiciones_laborales_insatisfactorias:
            trabajadores_abandonados += 1
            break

# Función para monitorear el proceso de ensamblaje
def monitorear_proceso(env):
    while True:
        yield env.timeout(60)  # Cada 60 minutos
        eficiencia = productos_ensamblados / (env.now / 60)  # Productos ensamblados por hora
        print(f'Tiempo: {env.now} minutos - Productos ensamblados: {productos_ensamblados} - Eficiencia: {eficiencia:.2f} productos/hora')

# Inicialización de trabajadores
for i in range(10):  # Supongamos 10 trabajadores
    env.process(ensamblar_producto(env, i))

# Inicialización del proceso de monitoreo
env.process(monitorear_proceso(env))

# Ejecución de la simulación
env.run(until=tiempo_total_simulacion)

# Resultados
print(f'\nProductos ensamblados en total: {productos_ensamblados}')
print(f'Trabajadores que abandonaron debido a condiciones laborales insatisfactorias: {trabajadores_abandonados}')


Tiempo: 60 minutos - Productos ensamblados: 20 - Eficiencia: 20.00 productos/hora
Tiempo: 120 minutos - Productos ensamblados: 36 - Eficiencia: 18.00 productos/hora
Tiempo: 180 minutos - Productos ensamblados: 41 - Eficiencia: 13.67 productos/hora
Tiempo: 240 minutos - Productos ensamblados: 41 - Eficiencia: 10.25 productos/hora
Tiempo: 300 minutos - Productos ensamblados: 41 - Eficiencia: 8.20 productos/hora
Tiempo: 360 minutos - Productos ensamblados: 41 - Eficiencia: 6.83 productos/hora
Tiempo: 420 minutos - Productos ensamblados: 41 - Eficiencia: 5.86 productos/hora

Productos ensamblados en total: 41
Trabajadores que abandonaron debido a condiciones laborales insatisfactorias: 10


Explicación:

Hemos simulado una línea de ensamblaje donde 10 trabajadores ensamblan productos electrónicos.
Cada trabajador toma un tiempo aleatorio para ensamblar un producto entre 10 y 30 minutos.
Existe una probabilidad del 20% de que un trabajador abandone debido a condiciones laborales insatisfactorias.
Hemos monitoreado la eficiencia del proceso cada 60 minutos y calculado el número de productos ensamblados por hora.
Al final de la simulación, hemos mostrado el número total de productos ensamblados y el número de trabajadores que abandonaron debido a condiciones laborales insatisfactorias.
Este ejemplo ilustra cómo se pueden usar simulaciones en Python para evaluar medidas de eficiencia, tiempos de servicio y la fuga de clientes (en este caso, trabajadores) en un contexto industrial de procesos. Las simulaciones como esta pueden ayudar a tomar decisiones informadas para mejorar la eficiencia y las condiciones de trabajo en la línea de ensamblaje.

# **Fórmula de Little en estado estacionario**

Puedes calcular L utilizando Python de la siguiente manera:

In [ ]:
# Datos del ejemplo
lambda_clientes_por_hora = 10  # Tasa de llegada de clientes por hora
w_tiempo_promedio_horas = 0.25  # Tiempo promedio que un cliente pasa en el mostrador (en horas)

# Calcular L usando la fórmula de Little
L = lambda_clientes_por_hora * w_tiempo_promedio_horas

# Imprimir el resultado
print(f"El número promedio de clientes en el mostrador de servicio es: {L} clientes")

El número promedio de clientes en el mostrador de servicio es: 2.5 clientes


Interpretación:

Hemos calculado la eficiencia de la línea de producción, que es el número promedio de unidades producidas por unidad de tiempo en comparación con la capacidad máxima teórica. En este caso, la eficiencia es el 93.33%, lo que sugiere que la línea de producción está funcionando a un nivel alto, pero no al máximo.

Luego, simulamos la producción durante 8 horas y calculamos el total de unidades producidas. Esto nos da una idea de la producción real en ese período.

Este ejercicio combina varios conceptos de la ingeniería industrial en un contexto práctico y muestra cómo se pueden aplicar y programar en Python para tomar decisiones informadas sobre la eficiencia de la producción en una fábrica.

Otro ejemplo, supongamos que tenemos una fábrica que produce productos electrónicos. Los trabajadores ensamblan componentes en productos terminados, y queremos medir el rendimiento del proceso en términos del número promedio de productos en el sistema y el tiempo promedio que un producto pasa en el sistema.

In [ ]:
import simpy
import random

# Configuración de la simulación
env = simpy.Environment()
tiempo_total_simulacion = 480  # Duración total de la simulación en minutos (8 horas)
productos_ensamblados = 0
tiempo_total_sistema = 0

# Parámetros del sistema
tiempo_ensamblaje_min = 10  # Tiempo mínimo para ensamblar un producto en minutos
tiempo_ensamblaje_max = 30  # Tiempo máximo para ensamblar un producto en minutos

# Función para el ensamblaje de productos
def ensamblar_producto(env):
    global productos_ensamblados, tiempo_total_sistema
    while True:
        tiempo_ensamblaje = random.uniform(tiempo_ensamblaje_min, tiempo_ensamblaje_max)
        yield env.timeout(tiempo_ensamblaje)
        productos_ensamblados += 1
        tiempo_total_sistema += env.now

# Inicialización de trabajadores
for i in range(10):  # Supongamos 10 trabajadores
    env.process(ensamblar_producto(env))

# Ejecución de la simulación
env.run(until=tiempo_total_simulacion)

# Cálculo de medidas de rendimiento
lambda_ = productos_ensamblados / tiempo_total_simulacion  # Tasa promedio de llegada de productos
W = tiempo_total_sistema / productos_ensamblados  # Tiempo promedio en el sistema
L = lambda_ * W  # Número promedio de productos en el sistema

# Resultados
print(f'Número promedio de productos en el sistema (L): {L:.2f}')
print(f'Tasa promedio de llegada de productos (λ): {lambda_:.2f} productos/minuto')
print(f'Tiempo promedio que un producto pasa en el sistema (W): {W:.2f} minutos')


Número promedio de productos en el sistema (L): 116.58
Tasa promedio de llegada de productos (λ): 0.49 productos/minuto
Tiempo promedio que un producto pasa en el sistema (W): 240.16 minutos


Hemos simulado una línea de ensamblaje donde 10 trabajadores ensamblan productos electrónicos. Cada trabajador toma un tiempo aleatorio para ensamblar un producto entre 10 y 30 minutos.

Después de ejecutar la simulación durante un período de tiempo (en este caso, 8 horas o 480 minutos), calculamos las siguientes medidas de rendimiento:

Número promedio de productos en el sistema (L): Usamos la Fórmula de Little (L=λ*W) para calcular el número promedio de productos en el sistema.

Tasa promedio de llegada de productos (λ): Calculamos la tasa promedio de llegada de productos dividiendo el número total de productos ensamblados por el tiempo total de simulación.

Tiempo promedio que un producto pasa en el sistema (W): Calculamos el tiempo promedio que un producto pasa en el sistema dividiendo el tiempo total en el sistema entre el número total de productos ensamblados.

Estas medidas de rendimiento nos permiten evaluar el rendimiento del proceso de ensamblaje en términos de la eficiencia y el tiempo que un producto pasa en el sistema.

### Ejercicio aplicado

Eres el gerente de una fábrica de productos electrónicos y estás tratando de mejorar la eficiencia de tu línea de producción. Tienes dos máquinas en serie, Máquina A y Máquina B, que ensamblan productos. Los productos llegan a una tasa promedio de 20 unidades por hora y se ensamblan en Máquina A, que tiene un tiempo promedio de servicio de 5 minutos por unidad. Luego, se envían a Máquina B, que tiene un tiempo promedio de servicio de 7 minutos por unidad. Quieres calcular la eficiencia de tu línea de producción y entender si hay un cuello de botella.

**Cálculos y Simulación en Python:**

Primero, calcularemos algunos valores clave y luego simularemos la producción durante un período de 8 horas.

In [ ]:
# Datos del ejercicio
tasa_llegada = 20  # Unidades por hora
tiempo_servicio_maquina_a = 2 #/ 60  # Tiempo de servicio de Máquina A en horas
tiempo_servicio_maquina_b = 3 #/ 60  # Tiempo de servicio de Máquina B en horas
horas_simulacion = 8

# Calcular la eficiencia de la línea de producción
lambda_unidades_por_hora = tasa_llegada
L_maquina_a = lambda_unidades_por_hora * tiempo_servicio_maquina_a
L_maquina_b = L_maquina_a * tiempo_servicio_maquina_b
eficiencia = L_maquina_b / (lambda_unidades_por_hora * horas_simulacion)

# Simulación de la producción durante 8 horas
produccion_total = 0
for hora in range(horas_simulacion):
    unidades_producidas_a = min(lambda_unidades_por_hora, L_maquina_a)
    L_maquina_a -= unidades_producidas_a
    unidades_producidas_b = min(unidades_producidas_a, L_maquina_b)
    L_maquina_b -= unidades_producidas_b
    produccion_total += unidades_producidas_b

# Imprimir resultados
print(f"La eficiencia de la línea de producción es: {eficiencia:.2%}")
print(f"Total de unidades producidas en 8 horas: {produccion_total} unidades")

La eficiencia de la línea de producción es: 75.00%
Total de unidades producidas en 8 horas: 40 unidades
